In [3]:
pip install gym pygame gym[other]

zsh:1: no matches found: gym[other]
Note: you may need to restart the kernel to use updated packages.


In [4]:
git clone https://github.com/tensorflow/tensorflow.git
cd tensorflow
./configure

SyntaxError: invalid syntax (3930454489.py, line 1)

In [ ]:
import gym
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Hiperparametreler
num_episodes = 100
eps = 1.0
eps_decay = 0.999
eps_min = 0.01

# Epsilon değerine dayalı olarak bir eylem seçer
def choose_action(state, eps):
    if np.random.uniform(0, 1) < eps:
        return env.action_space.sample()
    else:
        q_values = model.predict(state)
        return np.argmax(q_values[0])

# Gözlem verilerini modele uygun hale getirir
def get_state(observation):
    return np.array(observation).reshape(1, -1)

# OpenAI Gym'de CartPole oyununu başlat
env = gym.make('CartPole-v1')

# Modeli tanımla
model = Sequential()
model.add(Dense(64, input_shape=(4,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='linear'))
model.compile(loss='mse', optimizer=Adam(lr=0.001))

# Modeli eğit
for i in range(num_episodes):
    obs = env.reset()
    state = get_state(obs)
    done = False
    while not done:
        action = choose_action(state, eps)
        new_obs, reward, done, info = env.step(action)
        new_state = get_state(new_obs)
        target = model.predict(state)
        target[0][action] = reward + 0.95*np.max(model.predict(new_state)[0])
        model.fit(state, target, epochs=1, verbose=0)
        state = new_state
    eps = max(eps*eps_decay, eps_min)
    #print("Episode:", i+1, "Score:", info['score'])

# Modeli test et
obs = env.reset()
state = get_state(obs)
done = False
while not done:
    action = choose_action(state, eps_min)
    obs, reward, done, info = env.step(action)
    state = get_state(obs)
    env.render()


1/1 [==============================] - 0s 22ms/step


/usr/local/lib/python3.9/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


error: ignored

In [ ]:
!ls

sample_data


In [ ]:
import time
import gym
import gym_cap
import gym_cap.heuristic as policy
import numpy as np
import torch
import configparser

def change_ini_settings(file_path, section, key, new_value):
    config = configparser.ConfigParser()
    config.read(file_path)
    original_value = config.get(section, key)
    config.set(section, key, new_value)

    with open(file_path, 'w') as config_file:
        config.write(config_file)
    return original_value




device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("cap-v0")


output_file = open("results_1000_new.txt", "a")  # Metin dosyasını yazmak için dosya açılır
num_match = 1000
setting_file = 'setting.ini'

red_list=['Roomba'] #['Defense','Random','Zeros']
blue_list=['Defense','Patrol','Spiral','Random','Zeros']
values_to_change = {
    'elements': {
        'NUM_RED': '6',
        'NUM_RED_UAV': '3'
    },
    'settings': {
        'STOCH_TRANSITIONS': 'False',
        'STOCH_ATTACK': 'True'
    },
    'control': {
        'RED_ADV_BIAS': '2'
    },
    'memory': {
        'TEAM_MEMORY': 'fog',
        'INDIV_MEMORY': 'fog'
    },
    'communication': {
        'COM_GROUND': 'True',
        'COM_DISTANCE': '2'
    }
}

for section, key_value_dict in values_to_change.items():
    for key, new_value in key_value_dict.items():
        original_value = change_ini_settings(setting_file, section, key, new_value)

        output_file.write("time: " + time.ctime(time.time()) + " ( "+ key + " "+ original_value +" => "+ new_value +" )  \n")
        for bl in blue_list:
            for rl in red_list:
                observation = env.reset(
                                    map_size=20,
                                    config_path=setting_file,
                                    policy_red=getattr(policy, rl)(),
                                    policy_blue=getattr(policy, bl)() #policy.Fighter() # Defense Random Fighter Zeros Roomba Patrol Spiral Policy blue_policy
                                )
                action = env.action_space.sample()
                rscore = []
                start_time = time.time()

                for n in range(num_match):
                    done = False
                    rewards = []
                    while not done:
                        observation, reward, done, info = env.step(action)
                        rewards.append(reward)
                    env.reset()
                    rscore.append(sum(rewards))
                    duration = time.time() - start_time
                    #print("Time: %.2f s, Score: %.2f" % (duration, rscore[-1]))

                average_time = duration / num_match
                average_score = sum(rscore) / num_match
                print("red: "+rl+" blue: "+ bl+ " Average Time: %.2f s, Average Score: %.2f" % (duration/num_match, sum(rscore)/num_match))
                output_file.write("red: " + rl + " blue: " + bl + " Average Time: %.2f s, Average Score: %.2f\n" % (average_time, average_score))
        output_file.write("bitiş time: " + time.ctime(time.time()) + "\n")
        change_ini_settings(setting_file, section, key, original_value)


output_file.close()
env.close()

UnregisteredEnv: No registered env with id: cap-v0

In [ ]:
import gymnasium as gym
import gym_cap
from stable_baselines3 import SAC
from stable_baselines3.ppo.policies import MlpPolicy

#env = gym.make('CartPole-v1')
env = gym.make('cap-v0', disable_env_checker=True)

model = SAC(MlpPolicy, env, verbose=1)

In [ ]:
from stable_baselines3 import PPO
from gym_cap.envs import CapEnv

env = CapEnv()

models = []
for agent in env.get_team_blue() + env.get_team_red():
    model = PPO('MlpPolicy', env, verbose=1)
    models.append(model)

# Eğitim döngüsü
for model in models:
    model.learn(total_timesteps=10000)

In [ ]:
num_agents = len(env.get_team_blue) + len(env.get_team_red)
num_agents
models = [PPO("MlpPolicy", env, verbose=1) for _ in range(num_agents)]

In [ ]:
# fmt: off
"""
Training using REINFORCE for Mujoco
===================================

.. image:: /_static/img/tutorials/reinforce_invpend_gym_v26_fig1.gif
  :width: 400
  :alt: agent-environment-diagram

This tutorial serves 2 purposes:
 1. To understand how to implement REINFORCE [1] from scratch to solve Mujoco's InvertedPendulum-v4
 2. Implementation a deep reinforcement learning algorithm with Gymnasium's v0.26+ `step()` function

We will be using **REINFORCE**, one of the earliest policy gradient methods. Unlike going under the burden of learning a value function first and then deriving a policy out of it,
REINFORCE optimizes the policy directly. In other words, it is trained to maximize the probability of Monte-Carlo returns. More on that later.

**Inverted Pendulum** is Mujoco's cartpole but now powered by the Mujoco physics simulator -
which allows more complex experiments (such as varying the effects of gravity).
This environment involves a cart that can moved linearly, with a pole fixed on it at one end and having another end free.
The cart can be pushed left or right, and the goal is to balance the pole on the top of the cart by applying forces on the cart.
More information on the environment could be found at https://gymnasium.farama.org/environments/mujoco/inverted_pendulum/

**Training Objectives**: To balance the pole (inverted pendulum) on top of the cart

**Actions**: The agent takes a 1D vector for actions. The action space is a continuous ``(action)`` in ``[-3, 3]``,
where action represents the numerical force applied to the cart
(with magnitude representing the amount of force and sign representing the direction)

**Approach**: We use PyTorch to code REINFORCE from scratch to train a Neural Network policy to master Inverted Pendulum.

An explanation of the Gymnasium v0.26+ `Env.step()` function

``env.step(A)`` allows us to take an action 'A' in the current environment 'env'. The environment then executes the action
and returns five variables:

 - ``next_obs``: This is the observation that the agent will receive after taking the action.
 - ``reward``: This is the reward that the agent will receive after taking the action.
 - ``terminated``: This is a boolean variable that indicates whether or not the environment has terminated.
 - ``truncated``: This is a boolean variable that also indicates whether the episode ended by early truncation, i.e., a time limit is reached.
 - ``info``: This is a dictionary that might contain additional information about the environment.

"""
from __future__ import annotations

import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
from torch.distributions.normal import Normal

import gymnasium as gym


plt.rcParams["figure.figsize"] = (10, 5)


# %%
# Policy Network
# ~~~~~~~~~~~~~~
#
# .. image:: /_static/img/tutorials/reinforce_invpend_gym_v26_fig2.png
#
# We start by building a policy that the agent will learn using REINFORCE.
# A policy is a mapping from the current environment observation to a probability distribution of the actions to be taken.
# The policy used in the tutorial is parameterized by a neural network. It consists of 2 linear layers that are shared between both the predicted mean and standard deviation.
# Further, the single individual linear layers are used to estimate the mean and the standard deviation. ``nn.Tanh`` is used as a non-linearity between the hidden layers.
# The following function estimates a mean and standard deviation of a normal distribution from which an action is sampled. Hence it is expected for the policy to learn
# appropriate weights to output means and standard deviation based on the current observation.


class Policy_Network(nn.Module):
    """Parametrized Policy Network."""

    def __init__(self, obs_space_dims: int, action_space_dims: int):
        """Initializes a neural network that estimates the mean and standard deviation
         of a normal distribution from which an action is sampled from.

        Args:
            obs_space_dims: Dimension of the observation space
            action_space_dims: Dimension of the action space
        """
        super().__init__()

        hidden_space1 = 16  # Nothing special with 16, feel free to change
        hidden_space2 = 32  # Nothing special with 32, feel free to change

        # Shared Network
        self.shared_net = nn.Sequential(
            nn.Linear(obs_space_dims, hidden_space1),
            nn.Tanh(),
            nn.Linear(hidden_space1, hidden_space2),
            nn.Tanh(),
        )

        # Policy Mean specific Linear Layer
        self.policy_mean_net = nn.Sequential(
            nn.Linear(hidden_space2, action_space_dims)
        )

        # Policy Std Dev specific Linear Layer
        self.policy_stddev_net = nn.Sequential(
            nn.Linear(hidden_space2, action_space_dims)
        )

    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        """Conditioned on the observation, returns the mean and standard deviation
         of a normal distribution from which an action is sampled from.

        Args:
            x: Observation from the environment

        Returns:
            action_means: predicted mean of the normal distribution
            action_stddevs: predicted standard deviation of the normal distribution
        """
        shared_features = self.shared_net(x.float())

        action_means = self.policy_mean_net(shared_features)
        action_stddevs = torch.log(
            1 + torch.exp(self.policy_stddev_net(shared_features))
        )

        return action_means, action_stddevs


# %%
# Building an agent
# ~~~~~~~~~~~~~~~~~
#
# .. image:: /_static/img/tutorials/reinforce_invpend_gym_v26_fig3.jpeg
#
# Now that we are done building the policy, let us develop **REINFORCE** which gives life to the policy network.
# The algorithm of REINFORCE could be found above. As mentioned before, REINFORCE aims to maximize the Monte-Carlo returns.
#
# Fun Fact: REINFORCE is an acronym for " 'RE'ward 'I'ncrement 'N'on-negative 'F'actor times 'O'ffset 'R'einforcement times 'C'haracteristic 'E'ligibility
#
# Note: The choice of hyperparameters is to train a decently performing agent. No extensive hyperparameter
# tuning was done.
#


class REINFORCE:
    """REINFORCE algorithm."""

    def __init__(self, obs_space_dims: int, action_space_dims: int):
        """Initializes an agent that learns a policy via REINFORCE algorithm [1]
        to solve the task at hand (Inverted Pendulum v4).

        Args:
            obs_space_dims: Dimension of the observation space
            action_space_dims: Dimension of the action space
        """

        # Hyperparameters
        self.learning_rate = 1e-4  # Learning rate for policy optimization
        self.gamma = 0.99  # Discount factor
        self.eps = 1e-6  # small number for mathematical stability

        self.probs = []  # Stores probability values of the sampled action
        self.rewards = []  # Stores the corresponding rewards

        self.net = Policy_Network(obs_space_dims, action_space_dims)
        self.optimizer = torch.optim.AdamW(self.net.parameters(), lr=self.learning_rate)

    def sample_action(self, state: np.ndarray) -> float:
        """Returns an action, conditioned on the policy and observation.

        Args:
            state: Observation from the environment

        Returns:
            action: Action to be performed
        """
        state = torch.tensor(np.array([state]))
        action_means, action_stddevs = self.net(state)

        # create a normal distribution from the predicted
        #   mean and standard deviation and sample an action
        distrib = Normal(action_means[0] + self.eps, action_stddevs[0] + self.eps)
        action = distrib.sample()
        prob = distrib.log_prob(action)

        action = action.numpy()

        self.probs.append(prob)

        return action

    def update(self):
        """Updates the policy network's weights."""
        running_g = 0
        gs = []

        # Discounted return (backwards) - [::-1] will return an array in reverse
        for R in self.rewards[::-1]:
            running_g = R + self.gamma * running_g
            gs.insert(0, running_g)

        deltas = torch.tensor(gs)

        log_probs = torch.stack(self.probs)

        # Calculate the mean of log probabilities for all actions in the episode
        log_prob_mean = log_probs.mean()

        # Update the loss with the mean log probability and deltas
        # Now, we compute the correct total loss by taking the sum of the element-wise products.
        loss = -torch.sum(log_prob_mean * deltas)

        # Update the policy network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Empty / zero out all episode-centric/related variables
        self.probs = []
        self.rewards = []


# %%
# Now lets train the policy using REINFORCE to master the task of Inverted Pendulum.
#
# Following is the overview of the training procedure
#
#    for seed in random seeds
#        reinitialize agent
#
#        for episode in range of max number of episodes
#            until episode is done
#                sample action based on current observation
#
#                take action and receive reward and next observation
#
#                store action take, its probability, and the observed reward
#            update the policy
#
# Note: Deep RL is fairly brittle concerning random seed in a lot of common use cases (https://spinningup.openai.com/en/latest/spinningup/spinningup.html).
# Hence it is important to test out various seeds, which we will be doing.


# Create and wrap the environment
env = gym.make("InvertedPendulum-v4")
wrapped_env = gym.wrappers.RecordEpisodeStatistics(env, 50)  # Records episode-reward

total_num_episodes = int(5e3)  # Total number of episodes
# Observation-space of InvertedPendulum-v4 (4)
obs_space_dims = env.observation_space.shape[0]
# Action-space of InvertedPendulum-v4 (1)
action_space_dims = env.action_space.shape[0]
rewards_over_seeds = []

for seed in [1, 2, 3, 5, 8]:  # Fibonacci seeds
    # set seed
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    # Reinitialize agent every seed
    agent = REINFORCE(obs_space_dims, action_space_dims)
    reward_over_episodes = []

    for episode in range(total_num_episodes):
        # gymnasium v26 requires users to set seed while resetting the environment
        obs, info = wrapped_env.reset(seed=seed)

        done = False
        while not done:
            action = agent.sample_action(obs)

            # Step return type - `tuple[ObsType, SupportsFloat, bool, bool, dict[str, Any]]`
            # These represent the next observation, the reward from the step,
            # if the episode is terminated, if the episode is truncated and
            # additional info from the step
            obs, reward, terminated, truncated, info = wrapped_env.step(action)
            agent.rewards.append(reward)

            # End the episode when either truncated or terminated is true
            #  - truncated: The episode duration reaches max number of timesteps
            #  - terminated: Any of the state space values is no longer finite.
            done = terminated or truncated

        reward_over_episodes.append(wrapped_env.return_queue[-1])
        agent.update()

        if episode % 1000 == 0:
            avg_reward = int(np.mean(wrapped_env.return_queue))
            print("Episode:", episode, "Average Reward:", avg_reward)

    rewards_over_seeds.append(reward_over_episodes)


# %%
# Plot learning curve
# ~~~~~~~~~~~~~~~~~~~
#

rewards_to_plot = [[reward[0] for reward in rewards] for rewards in rewards_over_seeds]
df1 = pd.DataFrame(rewards_to_plot).melt()
df1.rename(columns={"variable": "episodes", "value": "reward"}, inplace=True)
sns.set(style="darkgrid", context="talk", palette="rainbow")
sns.lineplot(x="episodes", y="reward", data=df1).set(
    title="REINFORCE for InvertedPendulum-v4"
)
plt.show()

# %%
# .. image:: /_static/img/tutorials/reinforce_invpend_gym_v26_fig4.png
#
# Author: Siddarth Chandrasekar
#
# License: MIT License
#
# References
# ~~~~~~~~~~
#
# [1] Williams, Ronald J.. “Simple statistical gradient-following
# algorithms for connectionist reinforcement learning.” Machine Learning 8
# (2004): 229-256.
#


In [ ]:
import gymnasium as gym
import gym_cap  # CTF ortamını yüklemek için gereklidir
import numpy as np
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import deque
import gym_cap.heuristic as policy

class PolicyNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class REINFORCEAgent:
    def __init__(self, input_dim, output_dim, lr=1e-3):
        self.policy_network = PolicyNetwork(input_dim, output_dim)
        self.optimizer = torch.optim.Adam(self.policy_network.parameters(), lr=lr)
        self.log_probs = []
        self.rewards = []

    def select_action(self, state):
        state = state.reshape(-1)  # Gözlemi düzleştir
        state = torch.tensor(state, dtype=torch.float32)
        logits = self.policy_network(state)
        distribution = Categorical(logits=logits)
        action = distribution.sample()
        self.log_probs.append(distribution.log_prob(action))
        return action.item()

    def update_policy(self):
        discounted_rewards = []
        for t in range(len(self.rewards)):
            Gt = sum([r * (0.99 ** i) for i, r in enumerate(self.rewards[t:])])
            discounted_rewards.append(Gt)
        
        discounted_rewards = torch.tensor(discounted_rewards, dtype=torch.float32)
        policy_gradient = []
        for log_prob, Gt in zip(self.log_probs, discounted_rewards):
            policy_gradient.append(-log_prob * Gt)
        
        self.optimizer.zero_grad()
        policy_gradient = torch.stack(policy_gradient).sum()
        policy_gradient.backward()
        self.optimizer.step()
        
        self.log_probs = []
        self.rewards = []

def train(agent, env, num_episodes):
    episode_rewards = deque(maxlen=100)
    avg_rewards = []

    for episode in range(num_episodes):
        state = env.reset(policy_red=getattr(policy, 'Random')(),policy_blue=getattr(policy, 'Roomba')(),)
        done = False
        total_reward = 0

        while not done:
            action = agent.select_action(state)
            next_state, reward, done, _ = env.step(action)
            agent.rewards.append(reward)
            state = next_state
            total_reward += reward
        
        agent.update_policy()
        episode_rewards.append(total_reward)
        avg_reward = np.mean(episode_rewards)
        avg_rewards.append(avg_reward)

        if episode % 10 == 0:
            print(f'Episode {episode}, Average Reward: {avg_reward}')
    
    return avg_rewards

if __name__ == "__main__":
    env = gym.make("cap-v0", disable_env_checker=True)
    input_dim = 2400 #env.observation_space.shape[0]
    output_dim =  env.action_space.n

    agent = REINFORCEAgent(input_dim, output_dim)

    num_episodes = 50
    rewards = train(agent, env, num_episodes)

    # Plotting the learning curve
    plt.plot(rewards)
    plt.xlabel('Episodes')
    plt.ylabel('Average Reward')
    plt.title('REINFORCE Training for cap-v0')
    plt.show()

In [ ]:
len(rewards)

In [ ]:
env = gym.make("cap-v0", disable_env_checker=True)
obs = env.reset()
print("Gözlem uzayı boyutu:", obs.shape)

In [ ]:
# Veri setini oluştur
episodes = list(range(len(rewards)))
data = pd.DataFrame(list(zip(episodes, rewards)), columns=['Episode', 'Reward'])

# Seaborn ile öğrenme eğrisini çiz
sns.lineplot(x='Episode', y='Reward', data=data, estimator='mean')
plt.fill_between(data['Episode'], data['Reward'].rolling(2000, min_periods=100).min(),
                 data['Reward'].rolling(2000, min_periods=100).max(), color='blue', alpha=0.2)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('REINFORCE Training for cap-v0')
plt.show()

Test with PPO model

In [3]:
import gymnasium as gym
import gym_cap  # gym_cap environment
from stable_baselines3 import PPO, DQN, A2C
import matplotlib.pyplot as plt
import numpy as np
import gym_cap.heuristic as policy

""" from gym import spaces
class Board(spaces.Space):
    
    def __init__(self, _shape=None, dtype=np.uint8):
        assert dtype is not None, 'dtype must be explicitly provided. '
        self.dtype = np.dtype(dtype)

        if _shape is None:
            self.shape = (20, 20, NUM_CHANNEL)
        else:
            assert _shape[2] == NUM_CHANNEL
            self.shape = tuple(_shape)
        super(Board, self).__init__(self.shape, self.dtype)

    def __repr__(self):
        return "Board" + str(self.shape)

    def sample(self):
        map_obj = [NUM_BLUE, NUM_BLUE_UAV, NUM_RED, NUM_RED_UAV, NUM_GRAY]
        state, _, _ = gen_random_map('map',
                self.shape[0], rand_zones=False, map_obj=map_obj)
        return state"""


class CompatibleEnv(gym.Wrapper):
    def reset(self, **kwargs):
        # Ensure seed is not passed if not handled by the environment
        kwargs.pop('seed', None)

        # Reset and manage all returned values appropriately
        output = self.env.reset(**kwargs)
        if isinstance(output, tuple):
            # Only return the observation (assuming it's the first item)
            return output[0]
        return output  # Return directly if not a tuple
# Wrap your environment
enva = CompatibleEnv(gym.make('cap-v0'))
# Ortamı oluşturun
env = gym.make('cap-v0')
envx = gym.make('CartPole-v1')

# PPO modelini oluşturun
model = PPO("MlpPolicy", env, verbose=1)

# Eğitim adımları ve kazanma oranlarını kaydetmek için değişkenler
num_games = 50  # Toplam oyun sayısı
timesteps_per_game = 1000  # Her oyun için zaman adımı
win_rates = []
#model.learn(total_timesteps=timesteps_per_game)

# Modeli eğitin ve kazanma oranlarını kaydedin
obs = env.reset(
                    map_size=20,
                    #config_path='/demo/base_setting.ini',
                    policy_red=getattr(policy, 'Roomba')(),
                    policy_blue=getattr(policy, 'Zeros')() #policy.Fighter() # Defense Random Fighter Zeros Roomba Patrol Spiral Policy blue_policy
                )
for game in range(num_games):
    model.learn(total_timesteps=timesteps_per_game)
    rewards = []
    # Kazanma oranını hesaplayın
    
    wins = 0
    for _ in range(timesteps_per_game):
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(int(action))
        rewards.append(reward)
        if 'win' in info and info['win']:
            wins += 1
        if done:
            obs = env.reset()
    
    obs = env.reset()
    win_rate = sum(rewards) / timesteps_per_game  #wins
    win_rates.append(win_rate)
    print(f"Game {game + 1}/{num_games}: Win Rate = {win_rate:.2f}")

# Modeli kaydedin
model.save("ppo_ctf_model")

# Kazanma oranı grafiğini oluşturun
plt.figure(figsize=(10, 5))
plt.plot(np.arange(1, num_games + 1), win_rates, marker='o', linestyle='-', color='b')
plt.title('Win Rate per Game')
plt.xlabel('Game Number')
plt.ylabel('Win Rate')
plt.grid(True)
plt.show()

# Ortamı kapatın
env.close()


/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/envs/registration.py:481: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:189: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'numpy.ndarray'>`
  logger.warn(
Traceback (most recent

TypeError: reset() got an unexpected keyword argument 'seed'

In [ ]:
import gymnasium as gym
import gym_cap
import gym_cap.heuristic as policy
# Gym environment 'CartPole-v1' ile bir ortam oluşturuyoruz
env = gym.make('cap-v0')

# Ortamı başlat
#observation = env.reset()

# Toplam adım sayısı
total_steps = 1000
observation = env.reset(
                    map_size=20,
                    #config_path='/demo/base_setting.ini',
                    policy_red=getattr(policy, 'Roomba')(),
                    policy_blue=getattr(policy, 'Roomba')() #policy.Fighter() # Defense Random Fighter Zeros Roomba Patrol Spiral Policy blue_policy
                )
action = env.action_space.sample()
# Simülasyonu çalıştır
for step in range(total_steps):
    # Ortamı render et (opsiyonel)
    #action = env.action_space.sample()
    rewards = []
    done = False
    while not done:
        observation, reward, done, info = env.step(int(action))
        rewards.append(reward)
    env.reset()
    
    # Rastgele bir eylem seç
    #action = env.action_space.sample()
    
    # Eylemi gerçekleştirin ve yeni durumu alın
    #observation, reward, done, info = env.step(action)
    
    # Eğer epizot tamamlandıysa ortamı resetle
    if done:
        observation = env.reset()

# Ortamı kapat
env.close()

/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/envs/registration.py:481: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(
/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/Users/yakuponur/miniconda3/envs/asr/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:189: UserWarning: